In [1]:
import spacy
import timeit
import math
import pandas as pd
import shutil
import matplotlib.pyplot as plt
from os import path
from collections import Counter
from lxml import etree
from glob import glob
from unicodedata import normalize

In [2]:
nlp = spacy.load('fr_core_news_lg')

In [99]:
path_name_xml = r"/Users/jeanb/Documents/mini_memoire/en_cours/corpus_master/*.xml"
path_name_xml_selected = r"xml_selected/*.xml"

In [73]:
LEN_CHUNK = 1000
NB_BALANCE_CANON = 200
DATE_MIN = 1850
DATE_MAX = 1900

#### Let's retrieve 400 texts with canonical label balanced

In [4]:
def est_canon(tree):
    if tree.find(".//profileDesc") is not None:
        profil = tree.find(".//profileDesc")
        if profil.get("tag") == "canon":
            return True
        else:
            return False

In [78]:
def get_400_texts_balanced(path_name_xml, path_name_xml_selected, NB_BALANCE_CANON, DATE_MIN, DATE_MAX):
    nb_non_canon = 0
    nb_canon = 0
    for doc in glob(path_name_xml):
        with open(doc, encoding="utf8") as file:
        #on récupère l'arbre du document xml
            tree = etree.parse(file)
            tag = est_canon(tree)
            doc_name = path.splitext(path.basename(doc))[0]
            date = doc[59:].split('_')[0]
            if int(date) < DATE_MAX and int(date) >= DATE_MIN:
                if tag and nb_canon < NB_BALANCE_CANON:
                    shutil.copy(doc, path_name_xml_selected)
                    nb_canon+=1
                elif tag == False and nb_non_canon < NB_BALANCE_CANON:
                    shutil.copy(doc, path_name_xml_selected)
                    nb_non_canon+=1
    return

In [ ]:
get_400_texts_balanced(path_name_xml, path_name_xml_selected, NB_BALANCE_CANON, DATE_MIN, DATE_MAX)

#### Let's chunkise our 400 novels

In [80]:
path_folder_lemma = r"chunked_corpus_lemma_txt/"
path_folder_token = r"chunked_corpus_token_txt/"

In [11]:
def clean_text(txt):
    txt_res = normalize("NFKD", txt.replace('\xa0', ' '))
    txt_res = txt_res.replace('\\xa0', '')
    return txt_res

In [128]:
def chunkise_to_txt(list_token, doc_name, tag, LEN_CHUNK, path_folder):
    j = 0
    nb_chunk = 1
    for i in range(LEN_CHUNK, len(list_token), LEN_CHUNK):
        if len(list_token) - i > LEN_CHUNK:
            chunk = list_token[j:i]
            string = ' '.join(chunk)
            with open(r''+path_folder+doc_name+"_"+tag+'_CHUNK_'+str(nb_chunk)+".txt", "w", encoding="utf8") as file_out:
                file_out.write(string)
            j = i
            nb_chunk += 1

In [129]:
def pipeline_spacy(path):
    pos_ko = ["NUM", "X", "SYM", "PUNCT", "SPACE"]
    list_lemma = []
    list_token = []
    with open(path, 'r', encoding="utf8") as file:
        #on récupère l'arbre du document xml
        tree = etree.parse(file)
        #on récupère l'indice de canonicité du roman
        tag = est_canon(tree)
        if tag == True:
            indice = "canon"
            print("canon")
        else:
            indice = "noncanon"
            print("noncanon")
        #on récupère les paragraphes avec un xpath
        if tree.findall(".//p"):
            for paragraphe in tree.findall(".//p"):
                if paragraphe.text:
                    #on nettoie le texte et on le met dans la pipeline de spacy
                    text_clean = clean_text(str(paragraphe.text).lower())
                    docs = nlp(text_clean)
                    for token in docs:
                        #si le token est bien un mot on récupère son lemme
                        if token.pos_ not in pos_ko:
                            list_token.append(token.text)
                            list_lemma.append(token.lemma_)
                            
    return list_lemma, list_token, indice

In [133]:
def moulinette(path_name, LEN_CHUNK, path_folder_token, path_folder_lemma):
    i = 0
    for doc in glob(path_name):
        doc_name = path.splitext(path.basename(doc))[0]
        print(doc_name)
        print('---------- PIPELINE SPACY -----------')
        list_lemma, list_token, tag = pipeline_spacy(doc)
        #print(list_lemma)
        #print(list_token)
        print('---------- PIPELINE SPACY DONE -----------')

        print('---------- CHUNKIZE -----------')
        chunkise_to_txt(list_token, doc_name, tag, LEN_CHUNK, path_folder_token)
        chunkise_to_txt(list_lemma, doc_name, tag, LEN_CHUNK, path_folder_lemma)   
        print('---------- CHUNKIZE DONE -----------')
        
        i+=1
        print("PROGRESSION ", i/4,'% COMPLETED\n')


In [134]:
moulinette(path_name_xml_selected, LEN_CHUNK, path_folder_token, path_folder_lemma)

1850_Bassanville-Anais-de_Le-Soir-et-le-matin-de-la-vie
---------- PIPELINE SPACY -----------
noncanon
---------- PIPELINE SPACY DONE -----------
---------- CHUNKIZE -----------
---------- CHUNKIZE DONE -----------
PROGRESSION  0.25 % COMPLETED

1850_Dumas-Alexandre-Pere_Dieu-dispose-I
---------- PIPELINE SPACY -----------
canon
---------- PIPELINE SPACY DONE -----------
---------- CHUNKIZE -----------
---------- CHUNKIZE DONE -----------
PROGRESSION  0.5 % COMPLETED

1850_Dumas-Alexandre-Pere_Dieu-dispose-II
---------- PIPELINE SPACY -----------
canon
---------- PIPELINE SPACY DONE -----------
---------- CHUNKIZE -----------
---------- CHUNKIZE DONE -----------
PROGRESSION  0.75 % COMPLETED

1850_Dumas-Alexandre-Pere_La-colombe
---------- PIPELINE SPACY -----------
canon
---------- PIPELINE SPACY DONE -----------
---------- CHUNKIZE -----------
---------- CHUNKIZE DONE -----------
PROGRESSION  1.0 % COMPLETED

1850_Dumas-Alexandre-Pere_La-femme-au-collier-de-velours
---------- PIPELIN

---------- PIPELINE SPACY DONE -----------
---------- CHUNKIZE -----------
---------- CHUNKIZE DONE -----------
PROGRESSION  9.0 % COMPLETED

1853_Champfleury_Les-aventures-de-mademoiselle-Mariette
---------- PIPELINE SPACY -----------
noncanon
---------- PIPELINE SPACY DONE -----------
---------- CHUNKIZE -----------
---------- CHUNKIZE DONE -----------
PROGRESSION  9.25 % COMPLETED

1853_Chavannes-de-La-Giraudiere-Hippolyte-de_Les-Petits-voyageurs-en-Californie
---------- PIPELINE SPACY -----------
noncanon
---------- PIPELINE SPACY DONE -----------
---------- CHUNKIZE -----------
---------- CHUNKIZE DONE -----------
PROGRESSION  9.5 % COMPLETED

1853_Desnoiresterres-Gustave_Un-amour-en-diligence
---------- PIPELINE SPACY -----------
noncanon
---------- PIPELINE SPACY DONE -----------
---------- CHUNKIZE -----------
---------- CHUNKIZE DONE -----------
PROGRESSION  9.75 % COMPLETED

1853_Du-Camp-Maxime_Memoires-d'un-suicide
---------- PIPELINE SPACY -----------
noncanon
---------- PI

---------- PIPELINE SPACY DONE -----------
---------- CHUNKIZE -----------
---------- CHUNKIZE DONE -----------
PROGRESSION  17.75 % COMPLETED

1855_Dumas-Alexandre_La-derniere-annee-de-Marie-Dorval
---------- PIPELINE SPACY -----------
canon
---------- PIPELINE SPACY DONE -----------
---------- CHUNKIZE -----------
---------- CHUNKIZE DONE -----------
PROGRESSION  18.0 % COMPLETED

1855_Feval-Paul-(pere)_Oeuvres-de-Paul-Feval-soigneusement-revues-et-corrigees-L-Homme-de-fer-(1877)
---------- PIPELINE SPACY -----------
noncanon
---------- PIPELINE SPACY DONE -----------
---------- CHUNKIZE -----------
---------- CHUNKIZE DONE -----------
PROGRESSION  18.25 % COMPLETED

1855_Gerard-Jules_La-Chasse-au-lion
---------- PIPELINE SPACY -----------
noncanon
---------- PIPELINE SPACY DONE -----------
---------- CHUNKIZE -----------
---------- CHUNKIZE DONE -----------
PROGRESSION  18.5 % COMPLETED

1855_Nerval-Gerard-de_Aurelia
---------- PIPELINE SPACY -----------
canon
---------- PIPELINE SP

---------- PIPELINE SPACY DONE -----------
---------- CHUNKIZE -----------
---------- CHUNKIZE DONE -----------
PROGRESSION  26.0 % COMPLETED

1857_Dumas-Alexandre_Les-Compagnons-de-Jehu
---------- PIPELINE SPACY -----------
canon
---------- PIPELINE SPACY DONE -----------
---------- CHUNKIZE -----------
---------- CHUNKIZE DONE -----------
PROGRESSION  26.25 % COMPLETED

1857_Feuillet-Octave_La-petite-comtesse
---------- PIPELINE SPACY -----------
noncanon
---------- PIPELINE SPACY DONE -----------
---------- CHUNKIZE -----------
---------- CHUNKIZE DONE -----------
PROGRESSION  26.5 % COMPLETED

1857_Feval-Paul-(pere)_Le-Bossu
---------- PIPELINE SPACY -----------
noncanon
---------- PIPELINE SPACY DONE -----------
---------- CHUNKIZE -----------
---------- CHUNKIZE DONE -----------
PROGRESSION  26.75 % COMPLETED

1857_Feval-Paul-(pere)_Les-Compagnons-du-Silence-I
---------- PIPELINE SPACY -----------
noncanon
---------- PIPELINE SPACY DONE -----------
---------- CHUNKIZE -----------

---------- PIPELINE SPACY DONE -----------
---------- CHUNKIZE -----------
---------- CHUNKIZE DONE -----------
PROGRESSION  34.75 % COMPLETED

1859_Dumas-Alexandre_Le-fils-du-forçat
---------- PIPELINE SPACY -----------
canon
---------- PIPELINE SPACY DONE -----------
---------- CHUNKIZE -----------
---------- CHUNKIZE DONE -----------
PROGRESSION  35.0 % COMPLETED

1859_Dumas-Alexandre_Les-Louves-de-Machecoul
---------- PIPELINE SPACY -----------
canon
---------- PIPELINE SPACY DONE -----------
---------- CHUNKIZE -----------
---------- CHUNKIZE DONE -----------
PROGRESSION  35.25 % COMPLETED

1859_Feval-Paul-(pere)_La-Maison-de-Pilate
---------- PIPELINE SPACY -----------
noncanon
---------- PIPELINE SPACY DONE -----------
---------- CHUNKIZE -----------
---------- CHUNKIZE DONE -----------
PROGRESSION  35.5 % COMPLETED

1859_Feval-Paul-(pere)_Le-Roi-des-gueux
---------- PIPELINE SPACY -----------
noncanon
---------- PIPELINE SPACY DONE -----------
---------- CHUNKIZE -----------
--

...

---------- PIPELINE SPACY DONE -----------
---------- CHUNKIZE -----------
---------- CHUNKIZE DONE -----------
PROGRESSION  60.75 % COMPLETED

1864_Chevalier-H-Emile_Peaux-rouges-et-Peaux-blanches
---------- PIPELINE SPACY -----------
noncanon
---------- PIPELINE SPACY DONE -----------
---------- CHUNKIZE -----------
---------- CHUNKIZE DONE -----------
PROGRESSION  61.0 % COMPLETED

1864_Dumas-Alexandre_La-San-Felice
---------- PIPELINE SPACY -----------
canon
---------- PIPELINE SPACY DONE -----------
---------- CHUNKIZE -----------
---------- CHUNKIZE DONE -----------
PROGRESSION  61.25 % COMPLETED

1864_Erckmann-Chatrian_Histoire-d-un-conscrit-de-1813
---------- PIPELINE SPACY -----------
noncanon
---------- PIPELINE SPACY DONE -----------
---------- CHUNKIZE -----------
---------- CHUNKIZE DONE -----------
PROGRESSION  61.5 % COMPLETED

1864_Erckmann-Chatrian_L-ami-Fritz
---------- PIPELINE SPACY -----------
noncanon
---------- PIPELINE SPACY DONE -----------
---------- CHUNKIZE 

---------- PIPELINE SPACY DONE -----------
---------- CHUNKIZE -----------
---------- CHUNKIZE DONE -----------
PROGRESSION  69.5 % COMPLETED

1865_Verne-Jules_De-la-Terre-a-la-Lune
---------- PIPELINE SPACY -----------
canon
---------- PIPELINE SPACY DONE -----------
---------- CHUNKIZE -----------
---------- CHUNKIZE DONE -----------
PROGRESSION  69.75 % COMPLETED

1865_ Verne-Jules_Les-Forceurs-de-blocus
---------- PIPELINE SPACY -----------
canon
---------- PIPELINE SPACY DONE -----------
---------- CHUNKIZE -----------
---------- CHUNKIZE DONE -----------
PROGRESSION  70.0 % COMPLETED

1866_Chevalier-H-Emile_La-fille-des-indiens-rouges
---------- PIPELINE SPACY -----------
noncanon
---------- PIPELINE SPACY DONE -----------
---------- CHUNKIZE -----------
---------- CHUNKIZE DONE -----------
PROGRESSION  70.25 % COMPLETED

1866_Erckmann-Chatrian_Le-Blocus
---------- PIPELINE SPACY -----------
noncanon
---------- PIPELINE SPACY DONE -----------
---------- CHUNKIZE -----------
-----

---------- PIPELINE SPACY DONE -----------
---------- CHUNKIZE -----------
---------- CHUNKIZE DONE -----------
PROGRESSION  78.25 % COMPLETED

1872_Dumas-Alexandre-Pere_Le-docteur-mysterieux-I
---------- PIPELINE SPACY -----------
canon
---------- PIPELINE SPACY DONE -----------
---------- CHUNKIZE -----------
---------- CHUNKIZE DONE -----------
PROGRESSION  78.5 % COMPLETED

1872_Dumas-Alexandre-Pere_Le-docteur-mysterieux-II
---------- PIPELINE SPACY -----------
canon
---------- PIPELINE SPACY DONE -----------
---------- CHUNKIZE -----------
---------- CHUNKIZE DONE -----------
PROGRESSION  78.75 % COMPLETED

1872_Dumas-Alexandre_Robin-Hood-le-prince-des-voleurs-I
---------- PIPELINE SPACY -----------
canon
---------- PIPELINE SPACY DONE -----------
---------- CHUNKIZE -----------
---------- CHUNKIZE DONE -----------
PROGRESSION  79.0 % COMPLETED

1872_Sand-George_Francia
---------- PIPELINE SPACY -----------
canon
---------- PIPELINE SPACY DONE -----------
---------- CHUNKIZE -----

---------- PIPELINE SPACY DONE -----------
---------- CHUNKIZE -----------
---------- CHUNKIZE DONE -----------
PROGRESSION  87.5 % COMPLETED

1878_Zola-Emile_Madeleine-Ferat
---------- PIPELINE SPACY -----------
canon
---------- PIPELINE SPACY DONE -----------
---------- CHUNKIZE -----------
---------- CHUNKIZE DONE -----------
PROGRESSION  87.75 % COMPLETED

1878_Zola-Emile_Une-page-d-amour
---------- PIPELINE SPACY -----------
canon
---------- PIPELINE SPACY DONE -----------
---------- CHUNKIZE -----------
---------- CHUNKIZE DONE -----------
PROGRESSION  88.0 % COMPLETED

1879_Daudet-Alphonse_Lettres-de-mon-moulin
---------- PIPELINE SPACY -----------
canon
---------- PIPELINE SPACY DONE -----------
---------- CHUNKIZE -----------
---------- CHUNKIZE DONE -----------
PROGRESSION  88.25 % COMPLETED

1879_Goncourt-Edmond_Les-freres-Zemganno
---------- PIPELINE SPACY -----------
canon
---------- PIPELINE SPACY DONE -----------
---------- CHUNKIZE -----------
---------- CHUNKIZE DONE -

---------- PIPELINE SPACY DONE -----------
---------- CHUNKIZE -----------
---------- CHUNKIZE DONE -----------
PROGRESSION  96.75 % COMPLETED

1884_Guy-de-Maupassant_Au-soleil
---------- PIPELINE SPACY -----------
canon
---------- PIPELINE SPACY DONE -----------
---------- CHUNKIZE -----------
---------- CHUNKIZE DONE -----------
PROGRESSION  97.0 % COMPLETED

1884_Guy-de-Maupassant_Clair-de-lune
---------- PIPELINE SPACY -----------
canon
---------- PIPELINE SPACY DONE -----------
---------- CHUNKIZE -----------
---------- CHUNKIZE DONE -----------
PROGRESSION  97.25 % COMPLETED

1884_Guy-de-Maupassant_Les-soeurs-Rondoli
---------- PIPELINE SPACY -----------
canon
---------- PIPELINE SPACY DONE -----------
---------- CHUNKIZE -----------
---------- CHUNKIZE DONE -----------
PROGRESSION  97.5 % COMPLETED

1884_Guy-de-Maupassant_Miss-Harriet
---------- PIPELINE SPACY -----------
canon
---------- PIPELINE SPACY DONE -----------
---------- CHUNKIZE -----------
---------- CHUNKIZE DONE --